In [1]:
import numpy as np
import requests
import io

In [2]:
def robust_regression_grad(X, t, w, delta):
    """
    Compute the gradient of the average Huber (NOT squared error) loss for robust regression.

    Parameters
    ----------
        X: numpy array
            N x (D+1) numpy array for the train inputs (with dummy variables)
        t: numpy array
            N x 1 numpy array for the train targets
        w: numpy array
            (D+1) x 1 numpy array for the weights
        delta: positive float
            parameter for huber loss.


    Returns
    -------
        dw: numpy array
            (D+1) x 1 numpy array, the gradient of the huber loss in w
    """
    
    # Number of training examples
    N = X.shape[0]
    
    # Compute the residuals (r = Xw - t)
    residuals = X @ w - t
    
    # Initialize the gradient
    dw = np.zeros_like(w)
    
    # Iterate through each training example
    for i in range(N):
        r = residuals[i]
        
        if np.abs(r) <= delta:
            # Quadratic region: gradient is r * X[i]
            dw += r * X[i].reshape(-1, 1)
        else:
            # Linear region: gradient is delta * sign(r) * X[i]
            dw += delta * np.sign(r) * X[i].reshape(-1, 1)
    
    # Average the gradient over all training examples
    dw /= N
    
    return dw

In [3]:
def optimization(X, t, delta, lr, num_iterations=10000):
    """
    Compute (nearly) optimal weights for robust linear regression.

    Parameters
    ----------
        X: numpy array
            N x (D+1) numpy array for the train inputs (with dummy variables)
        t: numpy array
            N x 1 numpy array for the train targets
        delta: positive float
            parameter for huber loss.
        lr: positive float
            learning rate or step-size.

    Returns
    -------
        w: numpy array
            (D+1) x 1 numpy array, (nearly) optimal weights for robust linear regression
    """

    # some initialization for robust regression parameters
    w = np.zeros((X.shape[1], 1))
    
    for i in range(num_iterations):
        # Calculate predictions
        predictions = X @ w
        
        # Calculate residuals
        residuals = predictions - t
        
        # Compute the gradient of the Huber loss
        dw = robust_regression_grad(X, t, w, delta)
        
        # Update weights
        w -= lr * dw

    return w

In [4]:
def squared_error(y, t):
    """
    Compute the average squared error (NOT Huber) loss:
    
    sum_{i=1}^N (y^i - t^i)^2 / N

    Parameters
    ----------
        y: numpy array
            N x 1 numpy array for the predictions
        t: numpy array
            N x 1 numpy array for the train targets

    Returns
    -------
        cost: float
            the average squared error loss
    """
    dif = y - t
    L = 0.5 * np.power(dif, 2)
    return L.mean()

def linear_regression_optimal_weights(X, t):
    """
    Compute the optimal weights for linear regression.

    Parameters
    ----------
        X: numpy array
            N x (D+1) numpy array for the train inputs (with dummy variables)
        t: numpy array
            N x 1 numpy array for the train targets

    Returns
    -------
        w: numpy array
            (D+1) x 1 numpy array, optimal weights for linear regression
    """    
    w = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, t))
    return w


In [5]:
# load data
response = requests.get("https://www.cs.toronto.edu/~cmaddis/courses/sta314_f21/data/hw2_X.npy")
response.raise_for_status()
X = np.load(io.BytesIO(response.content))
response = requests.get("https://www.cs.toronto.edu/~cmaddis/courses/sta314_f21/data/hw2_t.npy")
response.raise_for_status()
t = np.load(io.BytesIO(response.content))
t = np.expand_dims(t, 1)
(N, D) = X.shape

# this code adds the "dummy variable"
ones_vector = np.ones((N, 1))
X = np.concatenate((ones_vector, X), axis=1)

# train, validation, test split using numpy indexing
X_train, X_val, X_test = X[30:], X[15:30], X[:15]
t_train, t_val, t_test = t[30:], t[15:30], t[:15]

lr = 0.01  # learning rate

# these are the deltas we will try for robust regression
deltas = [0.1, 0.5, 1, 5, 10]

# Report the validation squared error loss using standard linear regression
w_linreg = linear_regression_optimal_weights(X_train, t_train)
predictions = np.dot(X_val, w_linreg)
val_loss = squared_error(predictions, t_val)
print(f"linear regression validation loss: {val_loss}")

for i, delta in enumerate(deltas):

    # Optimize the parameters based on the train data for robust regression.
    w = optimization(X_train, t_train, delta, lr)

    # Report the validation and training squared error loss for this delta.
    val_pred = np.dot(X_val, w)
    val_loss = squared_error(t_val, val_pred)
    train_pred = np.dot(X_train, w)
    train_loss = squared_error(t_train, train_pred)
    print(f"delta: {delta}, valid. squared error loss: {val_loss}, train squared error loss: {train_loss}")

linear regression validation loss: 28.44202896790701
delta: 0.1, valid. squared error loss: 4.909228730093657, train squared error loss: 38.04144442529115
delta: 0.5, valid. squared error loss: 4.018972761905309, train squared error loss: 38.31116689103212
delta: 1, valid. squared error loss: 4.200164017924182, train squared error loss: 37.99343986336346
delta: 5, valid. squared error loss: 23.63610501876366, train squared error loss: 28.847567784752847
delta: 10, valid. squared error loss: 26.420467729491516, train squared error loss: 27.79829743150903
